In [1]:
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, TextVectorization
from pathlib import Path
from MetricsCallbackKeras import MetricsCallback
import psycopg2
import os
import numpy as np
import pandas as pd

In [2]:
conn = psycopg2.connect(database="mt_metrics",
                        host="localhost",
                        user="jonah",
                        password="getting started",
                        port=5432)
conn.autocommit = True

cursor = conn.cursor()

In [3]:
from time import time

import keras
from keras import layers
from keras.optimizers import RMSprop
# from keras import ops

import tensorflow as tf
from tensorflow import data as tf_data
import tensorflow_datasets as tfds
import tensorflow.data as tf_data


### Gradient Centralization for Better Training Performance
Author: Rishit Dagli

Description: Implement Gradient Centralization to improve training performance of DNNs.

https://keras.io/examples/vision/gradient_centralization/

First run is WITHOUT gradient centralization, second is WITH gradient centralization.

In [4]:
os.environ["MODEL_NAME"] = "Horses or Humans without Gradient Centralization"
os.environ["MODEL_TYPE"] = "RMSprop"

In [5]:
num_classes = 2
input_shape = (300, 300, 3)
dataset_name = "horses_or_humans"
batch_size = 128
AUTOTUNE = tf_data.AUTOTUNE

(train_ds, test_ds), metadata = tfds.load(
    name=dataset_name,
    split=[tfds.Split.TRAIN, tfds.Split.TEST],
    with_info=True,
    as_supervised=True,
)

print(f"Image shape: {metadata.features['image'].shape}")
print(f"Training images: {metadata.splits['train'].num_examples}")
print(f"Test images: {metadata.splits['test'].num_examples}")


Image shape: (300, 300, 3)
Training images: 1027
Test images: 256


In [6]:
rescale = layers.Rescaling(1.0 / 255)

data_augmentation = [
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.2),
]


# Helper to apply augmentation
def apply_aug(x):
    for aug in data_augmentation:
        x = aug(x)
    return x


def prepare(ds, shuffle=False, augment=False):
    # Rescale dataset
    ds = ds.map(lambda x, y: (rescale(x), y), num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1024)

    # Batch dataset
    ds = ds.batch(batch_size)

    # Use data augmentation only on the training set
    if augment:
        ds = ds.map(
            lambda x, y: (apply_aug(x), y),
            num_parallel_calls=AUTOTUNE,
        )

    # Use buffered prefecting
    return ds.prefetch(buffer_size=AUTOTUNE)


In [7]:
train_ds = prepare(train_ds, shuffle=True, augment=True)
test_ds = prepare(test_ds)


In [8]:
model = keras.Sequential(
    [
        layers.Input(shape=input_shape),
        layers.Conv2D(16, (3, 3), activation="relu"),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(32, (3, 3), activation="relu"),
        layers.Dropout(0.5),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.Dropout(0.5),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)


In [9]:
class GCRMSprop(RMSprop):
    def get_gradients(self, loss, params):
        # We here just provide a modified get_gradients() function since we are
        # trying to just compute the centralized gradients.

        grads = []
        gradients = super().get_gradients()
        for grad in gradients:
            grad_len = len(grad.shape)
            if grad_len > 1:
                axis = list(range(grad_len - 1))
                grad -= keras.keras.keras.keras.keras.keras.keras.keras.keras.ops.mean(grad, axis=axis, keep_dims=True)
            grads.append(grad)

        return grads


optimizer = GCRMSprop(learning_rate=1e-4)


In [10]:
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time() - self.epoch_time_start)


In [11]:
time_callback_no_gc = TimeHistory()
model.compile(
    loss="binary_crossentropy",
    optimizer=RMSprop(learning_rate=1e-4),
    metrics=["accuracy"],
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 149, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 dropout (Dropout)           (None, 147, 147, 32)      0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 73, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        1

In [12]:
for num_epochs in [5, 10, 15]:
    model.fit(
        train_ds, epochs=num_epochs, verbose=1, callbacks=[time_callback_no_gc, MetricsCallback(cursor)]
    )


Epoch 1/5
9/9 [==============================] - 16s 1s/step - loss: 0.7038 - accuracy: 0.4907
Epoch 2/5
9/9 [==============================] - 15s 1s/step - loss: 0.6784 - accuracy: 0.6008
Epoch 3/5
9/9 [==============================] - 14s 1s/step - loss: 0.6663 - accuracy: 0.6086
Epoch 4/5
9/9 [==============================] - 14s 1s/step - loss: 0.6623 - accuracy: 0.6514
Epoch 5/5
9/9 [==============================] - 15s 1s/step - loss: 0.6363 - accuracy: 0.6758
Epoch 1/10
9/9 [==============================] - 14s 1s/step - loss: 0.6135 - accuracy: 0.6699
Epoch 2/10
9/9 [==============================] - 13s 1s/step - loss: 0.6038 - accuracy: 0.6943
Epoch 3/10
9/9 [==============================] - 13s 1s/step - loss: 0.5816 - accuracy: 0.7361
Epoch 4/10
9/9 [==============================] - 14s 1s/step - loss: 0.5336 - accuracy: 0.7868
Epoch 5/10
9/9 [==============================] - 14s 1s/step - loss: 0.5415 - accuracy: 0.7575
Epoch 6/10
9/9 [=============================

In [13]:
os.environ["MODEL_NAME"] = "Horses or Humans with Gradient Centralization"
os.environ["MODEL_TYPE"] = "Gradient Centralization"

In [14]:
time_callback_gc = TimeHistory()
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

model.summary()

for num_epochs in [5, 10, 15]:
    history_gc = model.fit(train_ds, epochs=num_epochs, verbose=1, callbacks=[time_callback_gc, MetricsCallback(cursor)])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 149, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 dropout (Dropout)           (None, 147, 147, 32)      0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 73, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        1

### Simple MNIST convnet

Author: fchollet

Description: A simple convnet that achieves ~99% test accuracy on MNIST.

https://keras.io/examples/vision/mnist_convnet/

In [15]:
os.environ["MODEL_NAME"] = "Simple MNIST Convnet"
os.environ["MODEL_TYPE"] = "Categorical Crossentropy"

In [16]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [17]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dropout_3 (Dropout)         (None, 1600)             

In [18]:
batch_size = 128

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

for num_epochs in [10, 15, 20]:
    model.fit(x_train, y_train, batch_size=batch_size, epochs=num_epochs, validation_split=0.1, callbacks=[MetricsCallback(cursor)])


Epoch 1/10
422/422 [==============================] - 6s 15ms/step - loss: 0.3624 - accuracy: 0.8908 - val_loss: 0.0791 - val_accuracy: 0.9798
Epoch 2/10
422/422 [==============================] - 6s 15ms/step - loss: 0.1083 - accuracy: 0.9670 - val_loss: 0.0538 - val_accuracy: 0.9863
Epoch 3/10
422/422 [==============================] - 6s 14ms/step - loss: 0.0813 - accuracy: 0.9754 - val_loss: 0.0491 - val_accuracy: 0.9865
Epoch 4/10
422/422 [==============================] - 6s 14ms/step - loss: 0.0685 - accuracy: 0.9788 - val_loss: 0.0405 - val_accuracy: 0.9893
Epoch 5/10
422/422 [==============================] - 6s 15ms/step - loss: 0.0600 - accuracy: 0.9806 - val_loss: 0.0396 - val_accuracy: 0.9883
Epoch 6/10
422/422 [==============================] - 6s 15ms/step - loss: 0.0552 - accuracy: 0.9831 - val_loss: 0.0354 - val_accuracy: 0.9918
Epoch 7/10
422/422 [==============================] - 6s 14ms/step - loss: 0.0507 - accuracy: 0.9841 - val_loss: 0.0343 - val_accuracy: 0.9908

### Text classification from scratch

Authors: Mark Omernick, Francois Chollet

Description: Text sentiment classification starting from raw text files.

https://keras.io/examples/nlp/text_classification_from_scratch/

In [19]:
os.environ["MODEL_NAME"] = "Text classification from scratch"
os.environ["MODEL_TYPE"] = "Binary Crossentropy"

In [20]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  3764k      0  0:00:21  0:00:21 --:--:-- 3919k00:01  0:00:43 1862kM    0     0  3705k      0  0:00:22  0:00:16  0:00:06 3597k 3758k


In [21]:
!rm -r aclImdb/train/unsup

In [22]:
batch_size = 32
raw_train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")


Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


In [23]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

In [24]:
import string
import re


# Having looked at our data above, we see that the raw text contains HTML break
# tags of the form '<br />'. These tags will not be removed by the default
# standardizer (which doesn't strip HTML). Because of this, we will need to
# create a custom standardization function.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )


# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# Now that the vectorize_layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

# Let's make a text-only dataset (no labels):
text_ds = raw_train_ds.map(lambda x, y: x)
# Let's call `adapt`:
vectorize_layer.adapt(text_ds)


In [25]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)


In [26]:
# A integer input for vocab indices.
inputs = keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [27]:
# epochs = 3

# Fit the model using the train and test datasets.

for num_epochs in [3, 6, 9]:
    model.fit(train_ds, validation_data=val_ds, epochs=num_epochs, callbacks=[MetricsCallback(cursor)])


Epoch 1/3
625/625 [==============================] - 17s 26ms/step - loss: 0.5046 - accuracy: 0.7134 - val_loss: 0.3066 - val_accuracy: 0.8732
Epoch 2/3
625/625 [==============================] - 15s 25ms/step - loss: 0.2249 - accuracy: 0.9129 - val_loss: 0.3137 - val_accuracy: 0.8762
Epoch 3/3
625/625 [==============================] - 17s 26ms/step - loss: 0.1089 - accuracy: 0.9609 - val_loss: 0.3975 - val_accuracy: 0.8726
Epoch 1/6
625/625 [==============================] - 15s 24ms/step - loss: 0.0674 - accuracy: 0.9753 - val_loss: 0.4600 - val_accuracy: 0.8676
Epoch 2/6
625/625 [==============================] - 15s 25ms/step - loss: 0.0419 - accuracy: 0.9860 - val_loss: 0.5145 - val_accuracy: 0.8696
Epoch 3/6
625/625 [==============================] - 16s 25ms/step - loss: 0.0251 - accuracy: 0.9912 - val_loss: 0.6097 - val_accuracy: 0.8702
Epoch 4/6
625/625 [==============================] - 16s 26ms/step - loss: 0.0235 - accuracy: 0.9914 - val_loss: 0.7092 - val_accuracy: 0.8628

### Structured data classification with FeatureSpace

Author: fchollet

Description: Classify tabular data in a few lines of code.

https://keras.io/examples/structured_data/structured_data_classification_with_feature_space/

In [28]:
os.environ["MODEL_NAME"] = "FeatureSpace Structured Data Classification"
os.environ["MODEL_TYPE"] = "Imbalanced Classification"

In [29]:
from keras.utils import FeatureSpace

In [30]:
file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
dataframe = pd.read_csv(file_url)


In [31]:
val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)


Using 242 samples for training and 61 for validation


In [32]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("target")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)


In [33]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)


Input: {'age': <tf.Tensor: shape=(), dtype=int64, numpy=64>, 'sex': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'cp': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'trestbps': <tf.Tensor: shape=(), dtype=int64, numpy=140>, 'chol': <tf.Tensor: shape=(), dtype=int64, numpy=335>, 'fbs': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'restecg': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'thalach': <tf.Tensor: shape=(), dtype=int64, numpy=158>, 'exang': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'oldpeak': <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, 'slope': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'ca': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'thal': <tf.Tensor: shape=(), dtype=string, numpy=b'normal'>}
Target: tf.Tensor(0, shape=(), dtype=int64)


In [34]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)


In [35]:
feature_space = FeatureSpace(
    features={
        # Categorical features encoded as integers
        "sex": "integer_categorical",
        "cp": "integer_categorical",
        "fbs": "integer_categorical",
        "restecg": "integer_categorical",
        "exang": "integer_categorical",
        "ca": "integer_categorical",
        # Categorical feature encoded as string
        "thal": "string_categorical",
        # Numerical features to discretize
        "age": "float_discretized",
        # Numerical features to normalize
        "trestbps": "float_normalized",
        "chol": "float_normalized",
        "thalach": "float_normalized",
        "oldpeak": "float_normalized",
        "slope": "float_normalized",
    },
    # We create additional features by hashing
    # value co-occurrences for the
    # following groups of categorical features.
    crosses=[("sex", "age"), ("thal", "ca")],
    # The hashing space for these co-occurrences
    # wil be 32-dimensional.
    crossing_dim=32,
    # Our utility will one-hot encode all categorical
    # features and concat all features into a single
    # vector (one vector per sample).
    output_mode="concat",
)


In [36]:
feature_space = FeatureSpace(
    features={
        # Categorical features encoded as integers
        "sex": FeatureSpace.integer_categorical(num_oov_indices=0),
        "cp": FeatureSpace.integer_categorical(num_oov_indices=0),
        "fbs": FeatureSpace.integer_categorical(num_oov_indices=0),
        "restecg": FeatureSpace.integer_categorical(num_oov_indices=0),
        "exang": FeatureSpace.integer_categorical(num_oov_indices=0),
        "ca": FeatureSpace.integer_categorical(num_oov_indices=0),
        # Categorical feature encoded as string
        "thal": FeatureSpace.string_categorical(num_oov_indices=0),
        # Numerical features to discretize
        "age": FeatureSpace.float_discretized(num_bins=30),
        # Numerical features to normalize
        "trestbps": FeatureSpace.float_normalized(),
        "chol": FeatureSpace.float_normalized(),
        "thalach": FeatureSpace.float_normalized(),
        "oldpeak": FeatureSpace.float_normalized(),
        "slope": FeatureSpace.float_normalized(),
    },
    # Specify feature cross with a custom crossing dim.
    crosses=[
        FeatureSpace.cross(feature_names=("sex", "age"), crossing_dim=64),
        FeatureSpace.cross(
            feature_names=("thal", "ca"),
            crossing_dim=16,
        ),
    ],
    output_mode="concat",
)


In [37]:
train_ds_with_no_labels = train_ds.map(lambda x, _: x)
feature_space.adapt(train_ds_with_no_labels)


In [38]:
for x, _ in train_ds.take(1):
    preprocessed_x = feature_space(x)
    print("preprocessed_x.shape:", preprocessed_x.shape)
    print("preprocessed_x.dtype:", preprocessed_x.dtype)


preprocessed_x.shape: (32, 138)
preprocessed_x.dtype: <dtype: 'float32'>


In [39]:
preprocessed_train_ds = train_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
preprocessed_train_ds = preprocessed_train_ds.prefetch(tf.data.AUTOTUNE)

preprocessed_val_ds = val_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
preprocessed_val_ds = preprocessed_val_ds.prefetch(tf.data.AUTOTUNE)


In [40]:
dict_inputs = feature_space.get_inputs()
encoded_features = feature_space.get_encoded_features()

x = keras.layers.Dense(32, activation="relu")(encoded_features)
x = keras.layers.Dropout(0.5)(x)
predictions = keras.layers.Dense(1, activation="sigmoid")(x)

training_model = keras.Model(inputs=encoded_features, outputs=predictions)
training_model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)

inference_model = keras.Model(inputs=dict_inputs, outputs=predictions)


In [41]:

for num_epochs in [15, 20, 25]:
    training_model.fit(
        preprocessed_train_ds,
        epochs=num_epochs,
        validation_data=preprocessed_val_ds,
        verbose=2,
        callbacks=[MetricsCallback(cursor)]
    )


Epoch 1/15
8/8 - 0s - loss: 0.8196 - accuracy: 0.3802 - val_loss: 0.7108 - val_accuracy: 0.4918 - 427ms/epoch - 53ms/step
Epoch 2/15
8/8 - 0s - loss: 0.6957 - accuracy: 0.5537 - val_loss: 0.6522 - val_accuracy: 0.6557 - 149ms/epoch - 19ms/step
Epoch 3/15
8/8 - 0s - loss: 0.6741 - accuracy: 0.5826 - val_loss: 0.6043 - val_accuracy: 0.7377 - 149ms/epoch - 19ms/step
Epoch 4/15
8/8 - 0s - loss: 0.6203 - accuracy: 0.6488 - val_loss: 0.5655 - val_accuracy: 0.7705 - 148ms/epoch - 18ms/step
Epoch 5/15
8/8 - 0s - loss: 0.5775 - accuracy: 0.7066 - val_loss: 0.5331 - val_accuracy: 0.7541 - 146ms/epoch - 18ms/step
Epoch 6/15
8/8 - 0s - loss: 0.5569 - accuracy: 0.7479 - val_loss: 0.5075 - val_accuracy: 0.7377 - 153ms/epoch - 19ms/step
Epoch 7/15
8/8 - 0s - loss: 0.5273 - accuracy: 0.7397 - val_loss: 0.4836 - val_accuracy: 0.7377 - 151ms/epoch - 19ms/step
Epoch 8/15
8/8 - 0s - loss: 0.4978 - accuracy: 0.7479 - val_loss: 0.4655 - val_accuracy: 0.7541 - 148ms/epoch - 18ms/step
Epoch 9/15
8/8 - 0s - lo